<div class="alert alert-block alert-success">
<h1> Machine Learning - Project Cars4You</h1>
<h2> 1 - Exploratory Data Analysis and Data Preprocessing <h2>
<h2> Group 36 </h2>
<h3> 20225/2026 </h3>

***

## **Table of Contents**<br>

[1. **Importing libraries & Data**](#1-Importing-libraries-&-Data)<br>

[2. Modeling](#2-Modeling)<br>
- [R**2](#R**")<br>
- [MAE](#MAE")<br>
- [MSE](#MSE")<br>
- [RMSE](#RMSE")<br>
- [MedAE](#MedAE")<br>
- [MAPE](#MAPE")<br>
[2.1 Comparing Diferences](#2.1-Comparing-Diferences")<br>

[3. Final Predictions](#3-Final-Predictions)<br>
    

***

This notebook will consist of Modeling and making final predictions.

> Student Name - Artem Polikarpov
>> 20250443

> Student Name - Diogo Montenegro
>> 20250491

> Student Name - Francisco Martins
>> 20250482

> Student Name - João Cardoso
>> 20240529 


***

# 1. Import Libraries & Data

In [102]:
# general imports that we will need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
sns.set()

# filter methods
# - spearman 
# - chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import PredefinedSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error, root_mean_squared_error, mean_absolute_percentage_error
# embedded methods
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
# Scaler
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler
)

# ignore warnings
import warnings
warnings.filterwarnings('ignore')


#set random seed for reproducibility
RSEED = 42
np.random.seed(RSEED)
pd.set_option('display.max_rows', 500)


#### Data Loading

In [103]:
X_train_filtered = pd.read_csv('X_train_filtered_data.csv', index_col = 0)
y_train = pd.read_csv('y_train_data.csv', index_col=0)

X_val_filtered = pd.read_csv('X_val_filtered_data.csv', index_col=0)
y_val = pd.read_csv('y_val_data.csv', index_col=0)

test_filtered = pd.read_csv('test_filtered_data.csv', index_col=0)

In [104]:
X_train_filtered.head(2)

,mileage,tax,mpg,engineSize,carAge,model_te,Brand_audi,Brand_bmw,Brand_ford,Brand_hyundai,Brand_mercedes-benz,Brand_opel,Brand_toyota,Brand_volkswagen,transmission_automatic,transmission_manual,transmission_semi-auto,fuelType_diesel,fuelType_hybrid,fuelType_petrol
52404,-0.011088,-7.25,0.867133,-0.75,0.000000,-1.098325,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
65467,-0.120046,-6.25,0.251748,-0.50,0.333333,-0.823203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [105]:
cols_selected = X_train_filtered.columns.to_list()
cols_selected

['mileage',
 'tax',
 'mpg',
 'engineSize',
 'carAge',
 'model_te',
 'Brand_audi',
 'Brand_bmw',
 'Brand_ford',
 'Brand_hyundai',
 'Brand_mercedes-benz',
 'Brand_opel',
 'Brand_toyota',
 'Brand_volkswagen',
 'transmission_automatic',
 'transmission_manual',
 'transmission_semi-auto',
 'fuelType_diesel',
 'fuelType_hybrid',
 'fuelType_petrol']

# 2. Modeling

## R**2

In [106]:
#lr = LinearRegression().fit(X_train_filtered,y_train)

In [107]:
#rfr = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1).fit(X_train_filtered, y_train)
rfr = RandomForestRegressor(
    n_estimators=900,
    max_depth=25,        # let trees grow (or try 25)
    min_samples_leaf=3,    # was 10 → lower bias
    min_samples_split=6,
    max_features=0.8,      # use more features per split
    bootstrap=True,
    oob_score=True,
    random_state=42,
    n_jobs=-1
).fit(X_train_filtered, y_train)

In [108]:

#lasso = Lasso(alpha=0.001, random_state=42)
#lasso.fit(X_train_filtered, y_train)

In [109]:
y_pred_train = rfr.predict(X_train_filtered)
y_pred_val = rfr.predict(X_val_filtered)

In [110]:
r2_t = r2_score(y_train, y_pred_train)
r2_t

0.9721431559821373

In [111]:
r2_v = r2_score(y_val, y_pred_val)
r2_v

0.9466256538581055

In [112]:
r2 = r2_score(y_train, y_pred_train)
n = len(y_train)
p = len(X_train_filtered.columns)

def adj_r2 (r2,n,p):
    return 1-(1-r2)*(n-1)/(n-p-1)

ar2_t = adj_r2(r2,n,p)
ar2_t 

0.972133985155643

In [113]:
r2 = r2_score(y_val, y_pred_val)
n = len(y_val)
p = len(X_val_filtered.columns)

ar2_v = adj_r2(r2,n,p)
ar2_v

0.9465552994837011

## MAE


In [114]:
mae_t = mean_absolute_error(y_train, y_pred_train)
mae_t

894.6248631267458

In [115]:
mae_v = mean_absolute_error(y_val, y_pred_val)
mae_v

1316.419711571746

## MSE

In [116]:
mse_t = mean_squared_error(y_train, y_pred_train)
mse_t

2666893.8388144593

In [117]:
mse_v = mean_squared_error(y_val, y_pred_val)
mse_v

4863499.368342783

## RMSE

In [118]:
rmse_t = root_mean_squared_error(y_train, y_pred_train)
rmse_t

1633.0627173548662

In [119]:
rmse_v = root_mean_squared_error(y_val, y_pred_val)
rmse_v

2205.3342985458653

## MedAE

In [120]:
medae_t = median_absolute_error(y_train, y_pred_train)
medae_t

557.2497600938095

In [121]:
medae_v =median_absolute_error(y_val, y_pred_val)
medae_v

810.9841660478432

## MAPE

In [122]:
mape_t = mean_absolute_percentage_error(y_train, y_pred_train)
mape_t

0.05630743995581721

In [123]:
mape_v = mean_absolute_percentage_error(y_val, y_pred_val)
mape_v

0.08095129686262137

## Comparing differences

In [124]:
regression_metrics = pd.DataFrame({
    'Metric': ['R2', 'Adjusted R2', 'MAE', 'MSE', 'RMSE', 'MedAE', 'MAPE'],
    'Train': [r2_t, ar2_t, mae_t, mse_t, rmse_t, medae_t, mape_t],
    'Validation': [r2_v, ar2_v, mae_v, mse_v, rmse_v, medae_v, mape_v],
    }).round(2)

regression_metrics

,Metric,Train,Validation
0,R2,0.97,0.95
1,Adjusted R2,0.97,0.95
2,MAE,894.62,1316.42
3,MSE,2666893.84,4863499.37
4,RMSE,1633.06,2205.33
5,MedAE,557.25,810.98
6,MAPE,0.06,0.08


# 3. Final Predictions

In [125]:
test_filtered

,carID,mileage,tax,mpg,engineSize,carAge,model_te,Brand_audi,Brand_bmw,Brand_ford,...,Brand_mercedes-benz,Brand_opel,Brand_toyota,Brand_volkswagen,transmission_automatic,transmission_manual,transmission_semi-auto,fuelType_diesel,fuelType_hybrid,fuelType_petrol
0,89856,0.529862,3.00,-1.048951,0.000,-2.000000,-0.298456,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,106581,0.189617,0.25,-1.279720,0.500,0.000000,0.394323,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,80886,0.773717,-1.00,-0.356643,-0.125,0.333333,0.347736,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,100174,-0.477544,0.00,-0.867133,-0.500,-0.666667,-0.514390,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,81376,-0.336442,0.25,-0.356643,0.500,-0.666667,0.386308,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32562,105775,0.404771,0.00,-0.713287,-0.250,0.000000,0.394323,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
32563,81363,-0.619766,0.00,-1.573427,0.500,-1.000000,0.529976,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
32564,76833,-0.366904,0.00,-1.279720,0.500,-0.666667,0.920873,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
32565,91768,-0.471239,0.00,-1.958042,-0.375,-0.666667,0.002528,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [126]:
test_final = test_filtered.copy()

In [127]:
test_final

,carID,mileage,tax,mpg,engineSize,carAge,model_te,Brand_audi,Brand_bmw,Brand_ford,...,Brand_mercedes-benz,Brand_opel,Brand_toyota,Brand_volkswagen,transmission_automatic,transmission_manual,transmission_semi-auto,fuelType_diesel,fuelType_hybrid,fuelType_petrol
0,89856,0.529862,3.00,-1.048951,0.000,-2.000000,-0.298456,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,106581,0.189617,0.25,-1.279720,0.500,0.000000,0.394323,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,80886,0.773717,-1.00,-0.356643,-0.125,0.333333,0.347736,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,100174,-0.477544,0.00,-0.867133,-0.500,-0.666667,-0.514390,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,81376,-0.336442,0.25,-0.356643,0.500,-0.666667,0.386308,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32562,105775,0.404771,0.00,-0.713287,-0.250,0.000000,0.394323,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
32563,81363,-0.619766,0.00,-1.573427,0.500,-1.000000,0.529976,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
32564,76833,-0.366904,0.00,-1.279720,0.500,-0.666667,0.920873,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
32565,91768,-0.471239,0.00,-1.958042,-0.375,-0.666667,0.002528,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [128]:

test_final['price'] = rfr.predict(test_final[cols_selected])

#test_final['price'] = test_final['price'].round(0).astype(int)

#test_final['price'] = np.maximum(0, test_final['price'])


# test_final['price'] = test_final['price']


test_final['price'] = np.maximum(0, test_final['price'].round(0)).astype(int)

In [129]:
predictions = test_final[['carID', 'price']]

In [130]:
predictions.head()

,carID,price
0,89856,14505
1,106581,24161
2,80886,14194
3,100174,16499
4,81376,22204


In [131]:
predictions.to_csv("Group36_Version07.csv", index=False)


In [132]:
test_final

,carID,mileage,tax,mpg,engineSize,carAge,model_te,Brand_audi,Brand_bmw,Brand_ford,...,Brand_opel,Brand_toyota,Brand_volkswagen,transmission_automatic,transmission_manual,transmission_semi-auto,fuelType_diesel,fuelType_hybrid,fuelType_petrol,price
0,89856,0.529862,3.00,-1.048951,0.000,-2.000000,-0.298456,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,14505
1,106581,0.189617,0.25,-1.279720,0.500,0.000000,0.394323,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,24161
2,80886,0.773717,-1.00,-0.356643,-0.125,0.333333,0.347736,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,14194
3,100174,-0.477544,0.00,-0.867133,-0.500,-0.666667,-0.514390,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,16499
4,81376,-0.336442,0.25,-0.356643,0.500,-0.666667,0.386308,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,22204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32562,105775,0.404771,0.00,-0.713287,-0.250,0.000000,0.394323,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,17401
32563,81363,-0.619766,0.00,-1.573427,0.500,-1.000000,0.529976,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,35816
32564,76833,-0.366904,0.00,-1.279720,0.500,-0.666667,0.920873,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,34744
32565,91768,-0.471239,0.00,-1.958042,-0.375,-0.666667,0.002528,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,21633


In [133]:
predictions

,carID,price
0,89856,14505
1,106581,24161
2,80886,14194
3,100174,16499
4,81376,22204
...,...,...
32562,105775,17401
32563,81363,35816
32564,76833,34744
32565,91768,21633


In [134]:
with open("final_predictions.csv") as f:
    print(f.readline())


carID,price



In [136]:
print("Expected rows:", len(pd.read_csv("project_data/test.csv")))
print("Submissin rows:", len(predictions))


Expected rows: 32567
Submissin rows: 32567
